In [3]:
import pandas as pd
import numpy as np
from nfl_data_py import import_weekly_data, import_players, import_rosters, import_schedules
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt

# Global variables
seasons = list(range(2018, 2024))
target_cols = ['passing_yards', 'attempts', 'completions', 
               'rushing_yards', 'carries', 
               'receiving_yards', 'targets', 'receptions']
categorical_features = ['position_x', 'recent_team']
numerical_features = target_cols

# Function to load and preprocess data
def load_and_preprocess_data(seasons):
    player_stats = pd.concat([import_weekly_data(seasons)])
    players = import_players()
    df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')
    df = df.sort_values(['player_id', 'season', 'week'])

    for col in target_cols:
        df[f'next_week_{col}'] = df.groupby(['player_id', 'season'])[col].shift(-1)

    df = df.groupby(['player_id', 'season']).apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
    return df

# Function to update data with new weekly results
def update_data_with_new_week(current_df, new_week, current_season):
    new_data = import_weekly_data([current_season])
    new_data = new_data[new_data['week'] == new_week]
    updated_df = pd.concat([current_df, new_data], ignore_index=True)
    updated_df = updated_df.sort_values(['player_id', 'season', 'week'])

    for col in target_cols:
        updated_df[f'next_week_{col}'] = updated_df.groupby(['player_id', 'season'])[col].shift(-1)

    return updated_df

# Function to create and train models
def create_and_train_models(df):
    X = df[categorical_features + numerical_features]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_features)
        ])
    
    X_transformed = preprocessor.fit_transform(X)
    input_dim = X_transformed.shape[1]
    
    models = {}
    for stat in target_cols:
        y = df[f'next_week_{stat}'].values
        mask = ~np.isnan(y)
        X_stat = X_transformed[mask]
        y_stat = y[mask]
        
        model = Sequential([
            Input(shape=(input_dim,)),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(1, activation='linear')
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_stat, y_stat, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
        models[stat] = model
    
    return models, preprocessor

# Function to make predictions
def predict_next_week(player_name, df, models, preprocessor, current_week, current_season):
    try:
        latest_rosters = import_rosters([current_season])
        player_matches = latest_rosters[latest_rosters['player_name'].str.contains(player_name, case=False, na=False)]
        
        if player_matches.empty:
            print(f"Error: Player '{player_name}' not found in the latest roster.")
            return None
        
        if len(player_matches) > 1:
            print(f"Multiple matches found for '{player_name}'. Using the first match.")
        
        player_info = player_matches.iloc[0]
        
        player_stats = df[(df['player_id'] == player_info['player_id']) & 
                          (df['season'] == current_season) & 
                          (df['week'] == current_week)]
        
        if player_stats.empty:
            print(f"Error: No stats found for '{player_name}' in week {current_week}.")
            return None
        
        last_stats = player_stats.iloc[0]
        
        input_data = pd.DataFrame({col: [last_stats[col]] for col in numerical_features})
        input_data['position_x'] = [player_info['position']]
        input_data['recent_team'] = [player_info['team']]
        
        input_transformed = preprocessor.transform(input_data)
        
        predictions = {stat: models[stat].predict(input_transformed)[0][0] for stat in target_cols}
        
        print(f"Week {current_week + 1} Prediction for {player_name}:")
        print(f"Team: {player_info['team']}")
        print(f"Position: {player_info['position']}")
        
        if player_info['position'] in ['QB']:
            print(f"Predicted passing yards: {predictions['passing_yards']:.2f}")
            print(f"Predicted passing attempts: {predictions['attempts']:.2f}")
            print(f"Predicted passing completions: {predictions['completions']:.2f}")
            print(f"Predicted rushing yards: {predictions['rushing_yards']:.2f}")
            print(f"Predicted rushing attempts: {predictions['carries']:.2f}")
        elif player_info['position'] in ['RB']:
            print(f"Predicted rushing yards: {predictions['rushing_yards']:.2f}")
            print(f"Predicted rushing attempts: {predictions['carries']:.2f}")
            print(f"Predicted receiving yards: {predictions['receiving_yards']:.2f}")
            print(f"Predicted targets: {predictions['targets']:.2f}")
            print(f"Predicted receptions: {predictions['receptions']:.2f}")
        elif player_info['position'] in ['WR', 'TE']:
            print(f"Predicted receiving yards: {predictions['receiving_yards']:.2f}")
            print(f"Predicted targets: {predictions['targets']:.2f}")
            print(f"Predicted receptions: {predictions['receptions']:.2f}")
        
        return predictions
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Main execution flow
current_season = 2024
current_week = 0  # Start with week 0 (preseason)

# Initial data load and model training
df = load_and_preprocess_data(seasons)
models, preprocessor = create_and_train_models(df)

# Function to update models for a new week
def update_models_for_new_week(current_week, current_season):
    global df, models, preprocessor
    
    # Update data with new week's results
    df = update_data_with_new_week(df, current_week, current_season)
    
    # Retrain models with updated data
    models, preprocessor = create_and_train_models(df)
    
    print(f"Models updated with data from Week {current_week}")

# Example usage for the first few weeks of the season
for week in range(1):  # Weeks 1 to 3
    current_week = week
    
    # Update models with new week's data
    update_models_for_new_week(current_week, current_season)
    
    # Make predictions for the next week
    predict_next_week("Patrick Mahomes", df, models, preprocessor, current_week, current_season)
    predict_next_week("Christian McCaffrey", df, models, preprocessor, current_week, current_season)
    predict_next_week("Justin Jefferson", df, models, preprocessor, current_week, current_season)
    
    print("\n")

# You can continue this process for the entire season

Downcasting floats.


/tmp/ipykernel_17824/1304668421.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['player_id', 'season']).apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
/home/gin-g/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


HTTPError: HTTP Error 404: Not Found